In [1]:
import pandas as pd
import numpy as np

In [29]:
# --- Taking a look at the Russian River fish count data ---
raw=pd.read_excel("data/raw-data/russian.xlsx")

# drop subsistence harvest since there's only a couple years worth of data
data = raw.drop(['ER', 'LR'], axis = 1)
data.head()

,Year,Effort,Sport ER,Sport LR,Escape ER,Escape LR,Local ER,Local LR,total run
0,1962,ND,3408.0,1292.0,28980.0,19234.0,32388.0,20526.0,52914.0
1,1963,7880,3670.0,1390.0,14380.0,51120.0,18050.0,52510.0,70560.0
2,1964,5330,3550.0,2450.0,12700.0,46930.0,16250.0,49380.0,65630.0
3,1965,9720,10030.0,2160.0,21514.0,21820.0,31544.0,23980.0,55524.0
4,1966,18280,14950.0,7290.0,16658.0,34430.0,31608.0,41720.0,73328.0


In [3]:
totalrun = data["total run"].describe()
totalrun

count        59.000000
mean     140647.214451
std       65031.744582
min       38001.000000
25%       82314.500000
50%      140762.052632
75%      187981.000000
max      310225.000000
Name: total run, dtype: float64

In [18]:
# ---- CFS and gage height for Soldotna (station ID- 15266300) ----

#  This data contains CFS("discharge"), gauge height(feet), measurement quality, conditions at time of measurement
flowdfRAW = pd.read_csv("data/raw-data/surfaceWaterField-Soldotna", sep='\t', lineterminator='\n')
flowdfRAW.head()

,agency_cd,site_no,measurement_nu,measurement_dt,tz_cd,q_meas_used_fg,party_nm,site_visit_coll_agency_cd,gage_height_va,discharge_va,measured_rating_diff,gage_va_change,gage_va_time,control_type_cd,discharge_cd
0,USGS,15266300,1,1965-05-03 23:00:00,AKST,Yes,JTM/JPH,USGS,5.88,2340,Unspecified,0.01,0.8,NaN,NONE
1,USGS,15266300,2,1965-06-08 23:00:00,AKST,Yes,SHJ/G C,USGS,7.41,6110,Good,0.00,2.0,NaN,NONE
2,USGS,15266300,3,1965-07-21 23:00:00,AKST,Yes,G C,USGS,9.24,12100,Fair,-0.01,2.0,Clear,NONE
3,USGS,15266300,4,1965-08-24 23:00:00,AKST,Yes,G C/L H,USGS,8.68,10300,Good,-0.01,2.2,NaN,NONE
4,USGS,15266300,5,1965-09-23 23:00:00,AKST,Yes,G C,USGS,9.67,16000,Good,-0.02,2.0,NaN,NONE


In [28]:
# get a list of all columns with nulls
nanlist = [col for col in flowdfRAW.columns if flowdfRAW[col].isnull().any()]
nanlist

['tz_cd',
 'gage_height_va',
 'gage_va_change',
 'gage_va_time',
 'control_type_cd']

In [31]:
# drop the columns we don't want
flowdf = flowdfRAW.drop(columns=['agency_cd','site_no','measurement_nu', 'tz_cd', 'q_meas_used_fg', 'party_nm', 
                                 'site_visit_coll_agency_cd', 'discharge_cd', 'gage_va_change', 'gage_va_time'])
flowdf.isnull().sum()

measurement_dt           0
gage_height_va          43
discharge_va             0
measured_rating_diff     0
control_type_cd         47
dtype: int64

In [34]:
# rename the columns and check nulls 
new_col_names = ['date', 'gage_height(ft)', 'CFS', 'measurement_quality', 'measurement_conditions']
flowdf.rename(columns=dict(zip(flowdf.columns, new_col_names)), inplace=True)
flowdf.isnull().sum()


date                       0
gage_height(ft)           43
CFS                        0
measurement_quality        0
measurement_conditions    47
dtype: int64

In [41]:
# We must turn this column into pandas datetime. Here we also create two new columns for year and month, then we
# see how many entries we have for gage height, per year
flowdf['date'] = pd.to_datetime(flowdf['date'])
flowdf['year'], flowdf['month'] = flowdf['date'].dt.year, flowdf['date'].dt.month
flowdf.groupby('year')['gage_height(ft)'].count()


year
1965    7
1966    6
1967    4
1968    6
1969    6
1970    4
1971    3
1972    5
1973    5
1974    5
1975    5
1976    3
1977    6
1978    3
1979    3
1980    2
1981    2
1982    1
1983    5
1984    2
1985    3
1986    3
1987    4
1988    2
1989    0
1990    1
1991    0
1992    1
1993    1
1994    2
1995    6
1996    4
1997    6
1998    8
1999    7
2000    9
2001    9
2002    8
2003    6
2004    7
2005    8
2006    7
2007    8
2008    8
2009    8
2010    5
2011    8
2012    8
2013    9
2014    8
2015    6
2016    7
2017    5
2018    6
2019    8
Name: gage_height(ft), dtype: int64